In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

👆

为什么一定要换成`list`, 直接`dataset`不好吗?

In [6]:
connected_model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

numel_list = [p.numel()
              for p in connected_model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(3737474, [3145728, 1024, 524288, 512, 65536, 128, 256, 2])

就是:

$$
[3072 \times 1024, 1024 \times 512, 512 \times 128, 128 \times 2, 1024, 512, 128, 2]
$$


In [7]:
first_model = nn.Sequential(
                nn.Linear(3072, 512),
                nn.Tanh(),
                nn.Linear(512, 2),
                nn.LogSoftmax(dim=1))

numel_list = [p.numel() for p in first_model.parameters()]
sum(numel_list), numel_list

(1574402, [1572864, 512, 1024, 2])

👆

这两段说明参数很多, LOL

In [8]:
conv = nn.Conv2d(3, 16, kernel_size=3) # <1>
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

调用 `cpu()` 方法并不会改变原始张量的设备。它会返回一个新的张量，这个新张量在CPU上，而原始张量的设备不变。

所以，如果你在调用 `img.cpu()` 后再次调用 `img`，`img` 仍然在原来的设备上（在你的例子中，应该是CUDA设备）。

如果你想改变 `img` 的设备，你需要将 `cpu()` 的结果赋值给 `img`，如下所示：



这段代码创建了一个二维卷积层。

`nn.Conv2d(3, 16, kernel_size=3)` 这行代码的参数解释如下：

- `3` 是输入的通道数。在这个例子中，它是3，这意味着输入是一个彩色图像（红、绿、蓝三个通道）。
- `16` 是输出的通道数。这意味着这个卷积层将生成16个特征图（feature maps）。
- `kernel_size=3` 指定了卷积核的大小。在这个例子中，卷积核的大小是3x3。

输出 `Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))` 是这个卷积层的字符串表示。它告诉我们这个卷积层的一些参数：

- `3` 是输入的通道数。
- `16` 是输出的通道数。
- `kernel_size=(3, 3)` 是卷积核的大小。
- `stride=(1, 1)` 是卷积的步长。步长是卷积核在输入特征图上移动的步数。在这个例子中，步长是1，这意味着卷积核在水平和垂直方向上每次移动一步。



这样，`img` 就会在CPU上了。

这段代码创建了一个二维卷积层，这是在卷积神经网络（Convolutional Neural Networks，CNN）中常用的一种层。

`nn.Conv2d(3, 16, kernel_size=3)` 这行代码的参数解释如下：

- `3` 是输入的通道数。在这个例子中，它是3，这意味着输入是一个彩色图像（红、绿、蓝三个通道）。
- `16` 是输出的通道数。这意味着这个卷积层将生成16个特征图（feature maps）。
- `kernel_size=3` 指定了卷积核的大小。在这个例子中，卷积核的大小是3x3。

输出 `Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))` 是这个卷积层的字符串表示。它告诉我们这个卷积层的一些参数：

- `3` 是输入的通道数。
- `16` 是输出的通道数。
- `kernel_size=(3, 3)` 是卷积核的大小。
- `stride=(1, 1)` 是卷积的步长。步长是卷积核在输入特征图上移动的步数。在这个例子中，步长是1，这意味着卷积核在水平和垂直方向上每次移动一步。

这段代码创建了一个二维卷积层，这是在卷积神经网络（Convolutional Neural Networks，CNN）中常用的一种层。

`nn.Conv2d(3, 16, kernel_size=3)` 这行代码的参数解释如下：

- `3` 是输入的通道数。在这个例子中，它是3，这意味着输入是一个彩色图像（红、绿、蓝三个通道）。
- `16` 是输出的通道数。这意味着这个卷积层将生成16个特征图（feature maps）。
- `kernel_size=3` 指定了卷积核的大小。在这个例子中，卷积核的大小是3x3。

输出 `Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))` 是这个卷积层的字符串表示。它告诉我们这个卷积层的一些参数：

- `3` 是输入的通道数。
- `16` 是输出的通道数。
- `kernel_size=(3, 3)` 是卷积核的大小。
- `stride=(1, 1)` 是卷积的步长。步长是卷积核在输入特征图上移动的步数。在这个例子中，步长是1，这意味着卷积核在水平和垂直方向上每次移动一步。

这段代码创建了一个二维卷积层，这是在卷积神经网络（Convolutional Neural Networks，CNN）中常用的一种层。

`nn.Conv2d(3, 16, kernel_size=3)` 这行代码的参数解释如下：

- `3` 是输入的通道数。在这个例子中，它是3，这意味着输入是一个彩色图像（红、绿、蓝三个通道）。
- `16` 是输出的通道数。这意味着这个卷积层将生成16个特征图（feature maps）。
- `kernel_size=3` 指定了卷积核的大小。在这个例子中，卷积核的大小是3x3。

输出 `Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))` 是这个卷积层的字符串表示。它告诉我们这个卷积层的一些参数：

- `3` 是输入的通道数。
- `16` 是输出的通道数。
- `kernel_size=(3, 3)` 是卷积核的大小。
- `stride=(1, 1)` 是卷积的步长。步长是卷积核在输入特征图上移动的步数。在这个例子中，步长是1，这意味着卷积核在水平和垂直方向上每次移动一步。

这段代码创建了一个二维卷积层，这是在卷积神经网络（Convolutional Neural Networks，CNN）中常用的一种层。

`nn.Conv2d(3, 16, kernel_size=3)` 这行代码的参数解释如下：

- `3` 是输入的通道数。在这个例子中，它是3，这意味着输入是一个彩色图像（红、绿、蓝三个通道）。
- `16` 是输出的通道数。这意味着这个卷积层将生成16个特征图（feature maps）。
- `kernel_size=3` 指定了卷积核的大小。在这个例子中，卷积核的大小是3x3。

输出 `Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))` 是这个卷积层的字符串表示。它告诉我们这个卷积层的一些参数：

- `3` 是输入的通道数。
- `16` 是输出的通道数。
- `kernel_size=(3, 3)` 是卷积核的大小。
- `stride=(1, 1)` 是卷积的步长。步长是卷积核在输入特征图上移动的步数。在这个例子中，步长是1，这意味着卷积核在水平和垂直方向上每次移动一步。

这段代码创建了一个二维卷积层，这是在卷积神经网络（Convolutional Neural Networks，CNN）中常用的一种层。

`nn.Conv2d(3, 16, kernel_size=3)` 这行代码的参数解释如下：

- `3` 是输入的通道数。在这个例子中，它是3，这意味着输入是一个彩色图像（红、绿、蓝三个通道）。
- `16` 是输出的通道数。这意味着这个卷积层将生成16个特征图（feature maps）。
- `kernel_size=3` 指定了卷积核的大小。在这个例子中，卷积核的大小是3x3。

输出 `Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))` 是这个卷积层的字符串表示。它告诉我们这个卷积层的一些参数：

- `3` 是输入的通道数。
- `16` 是输出的通道数。
- `kernel_size=(3, 3)` 是卷积核的大小。
- `stride=(1, 1)` 是卷积的步长。步长是卷积核在输入特征图上移动的步数。在这个例子中，步长是1，这意味着卷积核在水平和垂直方向上每次移动一步。